In [ ]:
import numpy as np
import pandas as pd
from cvxopt import matrix, solvers

In [ ]:
df = pd.read_csv('Data_DJIA.csv', index_col = 1)
df['ADJ_PRC'] =  df['PRC']/df['CFACPR']
price = df.pivot(columns = 'PERMNO', values = 'ADJ_PRC')
df
price

PERMNO,11308,11703,14322,14541,17830,18163,19561,22592,24643,59176
date,,,,,,,,,,
19791231,0.705259,6.385909,6.938935,7.046875,2.687500,2.301340,5.000000,6.054503,7.716799,1.932482
19800131,0.718036,6.445221,6.601626,7.781250,3.117188,2.235476,6.469136,5.964137,8.929690,1.900139
19800229,0.672041,6.010267,6.312503,9.937500,2.914062,2.208356,5.950617,6.024381,9.386722,1.811197
19800331,0.643932,5.476461,6.264316,8.187500,2.578125,2.099876,5.259259,5.994259,8.191409,1.908225
19800430,0.679706,5.773020,6.216129,8.843750,2.632812,2.216105,5.407407,6.355722,7.734377,2.183139
...,...,...,...,...,...,...,...,...,...,...
19890831,4.156250,18.643691,17.491899,14.062500,6.968750,7.965580,18.250000,19.037044,21.656259,9.631202
19890929,4.187500,18.940251,16.287223,15.250000,6.953125,7.438674,19.000000,18.103265,21.585947,9.371776
19891031,4.523438,18.188967,14.841611,17.062500,6.671875,8.004323,18.750000,17.259851,19.687509,9.436632


In [ ]:
code_dict = {24643: 'Alcoa',
             59176: 'Amex',
             19561: 'Boeing',
             14541: 'Chev',
             11308: 'Coke',
             11703: 'Du Pont',
             22592: 'MMM',
             18163: 'P&G',
             14322: 'Sears',
             17830: 'U Tech'}

In [ ]:
price.rename(columns = code_dict, inplace = True)
price = price[code_dict.values()]
price

PERMNO,Alcoa,Amex,Boeing,Chev,Coke,Du Pont,MMM,P&G,Sears,U Tech
date,,,,,,,,,,
19791231,7.716799,1.932482,5.000000,7.046875,0.705259,6.385909,6.054503,2.301340,6.938935,2.687500
19800131,8.929690,1.900139,6.469136,7.781250,0.718036,6.445221,5.964137,2.235476,6.601626,3.117188
19800229,9.386722,1.811197,5.950617,9.937500,0.672041,6.010267,6.024381,2.208356,6.312503,2.914062
19800331,8.191409,1.908225,5.259259,8.187500,0.643932,5.476461,5.994259,2.099876,6.264316,2.578125
19800430,7.734377,2.183139,5.407407,8.843750,0.679706,5.773020,6.355722,2.216105,6.216129,2.632812
...,...,...,...,...,...,...,...,...,...,...
19890831,21.656259,9.631202,18.250000,14.062500,4.156250,18.643691,19.037044,7.965580,17.491899,6.968750
19890929,21.585947,9.371776,19.000000,15.250000,4.187500,18.940251,18.103265,7.438674,16.287223,6.953125
19891031,19.687509,9.436632,18.750000,17.062500,4.523438,18.188967,17.259851,8.004323,14.841611,6.671875


In [ ]:
rtn = price.pct_change()[1:] # change form 2 to 1
rtn


PERMNO,Alcoa,Amex,Boeing,Chev,Coke,Du Pont,MMM,P&G,Sears,U Tech
date,,,,,,,,,,
19800131,0.157175,-0.016736,0.293827,0.104213,0.018116,0.009288,-0.014925,-0.028620,-0.048611,0.159884
19800229,0.051181,-0.046809,-0.080153,0.277108,-0.064057,-0.067485,0.010101,-0.012132,-0.043796,-0.065163
19800331,-0.127341,0.053571,-0.116183,-0.176101,-0.041825,-0.088816,-0.005000,-0.049123,-0.007634,-0.115282
19800430,-0.055794,0.144068,0.028169,0.080153,0.055556,0.054152,0.060302,0.055351,-0.007692,0.021212
19800530,0.084091,0.007407,-0.058219,0.033569,0.015038,0.078767,0.028436,0.089161,0.031008,0.014837
...,...,...,...,...,...,...,...,...,...,...
19890831,0.115942,-0.003356,0.045346,0.032110,-0.022059,0.036264,0.039474,0.076440,-0.026810,0.015945
19890929,-0.003247,-0.026936,0.041096,0.084444,0.007519,0.015907,-0.049051,-0.066148,-0.068871,-0.002242
19891031,-0.087948,0.006920,-0.013158,0.118852,0.080224,-0.039666,-0.046589,0.076042,-0.088757,-0.040449


In [ ]:
# mean = rtn.mean().values
# std = rtn.std().values
# corr = rtn.corr().values
# cov = rtn.cov().values
mean = np.array([.015617, .019477, .01907, .015801, .021643, .016010, .014892, .016248,.014075, .014537])
std = np.array([.088308, .084585, .01004, .086215, .059886, .068767, .058162, .056385, .080047, .082125])
corr = np.array([[ 1,  0.366, 0.3457,   0.1606,     0.2279,  0.5133,  0.5203,    0.2176,  0.3267,  0.5101], 
             [ 0.366,  1,  0.5379,    0.2165,  0.4986,   0.5823,  0.5569,    0.4760,  0.6517,  0.5853],
             [0.3457,  0.5379,   1,     0.2218,  0.4283,  0.4051,  0.4492,   0.3867,  0.4883,  0.6569],
             [0.1606,  0.2165,   0.2218, 1,    0.0569,  0.3609,  0.2325,    0.2289,  0.1726,  0.3814 ],
             [0.2279,  0.4986,   0.4283, 0.0569,  1,     0.3619,  0.4811,    0.5952,  0.4378,  0.4368],
             [0.5133,  0.5823,   0.4051, 0.3609,  0.3619,  1,     0.6167,    0.4996,  0.5811,  0.5644],
             [0.5203,  0.5569,   0.4492, 0.2325,  0.4811,  0.6167,  1,      0.6037,  0.5671,  0.6032 ],
             [0.2176,  0.4760,   0.3867, 0.2289,  0.5952,  0.4996,  0.6037,      1,  0.5012,  0.4772 ],
             [0.3267,  0.6517,   0.4883, 0.1726,  0.4378,  0.5811,  0.5671,    0.5012,  1,    0.6039 ],
             [ 0.5101, 0.5853,   0.6569, 0.3814,  0.4368,  0.5644,  0.6032,    0.4772,  0.6039,   1 ]])

# mean = np.array([0.01218968, 0.01650123, 0.01618233, 0.01080502, 0.01789969,
#        0.01165278, 0.01135932, 0.01272683, 0.00944854, 0.01125249])
# std = np.array([0.08732951, 0.08428415, 0.09998456, 0.0850044 , 0.0598025 ,
#        0.06815065, 0.05772651, 0.05636781, 0.08001666, 0.08200148])
# corr = np.array([[1.        , 0.37399317, 0.33449093, 0.14209886, 0.2420464 ,
#         0.50008218, 0.50805394, 0.22515081, 0.31456661, 0.49966064],
#        [0.37399317, 1.        , 0.53830496, 0.21985643, 0.50444258,
#         0.58574141, 0.56127812, 0.4820687 , 0.64883383, 0.58984795],
#        [0.33449093, 0.53830496, 1.        , 0.20728195, 0.44187654,
#         0.40164498, 0.4477631 , 0.38062161, 0.49076697, 0.65681071],
#        [0.14209886, 0.21985643, 0.20728195, 1.        , 0.05731009,
#         0.33375102, 0.21787283, 0.21623184, 0.15258004, 0.37337866],
#        [0.2420464 , 0.50444258, 0.44187654, 0.05731009, 1.        ,
#         0.37855122, 0.4883554 , 0.5987313 , 0.438786  , 0.44596156],
#        [0.50008218, 0.58574141, 0.40164498, 0.33375102, 0.37855122,
#         1.        , 0.60858385, 0.49674194, 0.58039335, 0.55817796],
#        [0.50805394, 0.56127812, 0.4477631 , 0.21787283, 0.4883554 ,
#         0.60858385, 1.        , 0.60612648, 0.56276084, 0.60204987],
#        [0.22515081, 0.4820687 , 0.38062161, 0.21623184, 0.5987313 ,
#         0.49674194, 0.60612648, 1.        , 0.49008901, 0.46808594],
#        [0.31456661, 0.64883383, 0.49076697, 0.15258004, 0.438786  ,
#         0.58039335, 0.56276084, 0.49008901, 1.        , 0.60259957],
#        [0.49966064, 0.58984795, 0.65681071, 0.37337866, 0.44596156,
#         0.55817796, 0.60204987, 0.46808594, 0.60259957, 1.        ]])
cov = np.zeros((10, 10))
for i in range(10):
  for j in range(10):
    cov[i][j] = corr[i][j]*std[i]*std[j]
cov

array([[0.0077983 , 0.00273385, 0.0003065 , 0.00122272, 0.00120523,
        0.0031171 , 0.00267235, 0.00108348, 0.00230937, 0.0036994 ],
       [0.00273385, 0.00715462, 0.0004568 , 0.00157883, 0.00252564,
        0.00338704, 0.00273974, 0.0022702 , 0.00441251, 0.00406581],
       [0.0003065 , 0.0004568 , 0.0001008 , 0.00019199, 0.00025752,
        0.00027969, 0.00026231, 0.00021891, 0.00039243, 0.00054164],
       [0.00122272, 0.00157883, 0.00019199, 0.00743303, 0.00029378,
        0.00213968, 0.00116586, 0.00111274, 0.00119116, 0.00270047],
       [0.00120523, 0.00252564, 0.00025752, 0.00029378, 0.00358633,
        0.00149037, 0.00167571, 0.0020098 , 0.00209868, 0.00214824],
       [0.0031171 , 0.00338704, 0.00027969, 0.00213968, 0.00149037,
        0.0047289 , 0.00246657, 0.00193716, 0.00319872, 0.00318744],
       [0.00267235, 0.00273974, 0.00026231, 0.00116586, 0.00167571,
        0.00246657, 0.00338282, 0.00197981, 0.00264024, 0.00288122],
       [0.00108348, 0.0022702 , 0.0002189

In [ ]:
rtn.mean()

PERMNO
Alcoa      0.012190
Amex       0.016501
Boeing     0.016182
Chev       0.010805
Coke       0.017900
Du Pont    0.011653
MMM        0.011359
P&G        0.012727
Sears      0.009449
U Tech     0.011252
dtype: float64

In [ ]:
rtn.std()

PERMNO
Alcoa      0.087330
Amex       0.084284
Boeing     0.099985
Chev       0.085004
Coke       0.059802
Du Pont    0.068151
MMM        0.057727
P&G        0.056368
Sears      0.080017
U Tech     0.082001
dtype: float64

In [ ]:
rtn.corr()

PERMNO,Alcoa,Amex,Boeing,Chev,Coke,Du Pont,MMM,P&G,Sears,U Tech
PERMNO,,,,,,,,,,
Alcoa,1.000000,0.373993,0.334491,0.142099,0.242046,0.500082,0.508054,0.225151,0.314567,0.499661
Amex,0.373993,1.000000,0.538305,0.219856,0.504443,0.585741,0.561278,0.482069,0.648834,0.589848
Boeing,0.334491,0.538305,1.000000,0.207282,0.441877,0.401645,0.447763,0.380622,0.490767,0.656811
Chev,0.142099,0.219856,0.207282,1.000000,0.057310,0.333751,0.217873,0.216232,0.152580,0.373379
Coke,0.242046,0.504443,0.441877,0.057310,1.000000,0.378551,0.488355,0.598731,0.438786,0.445962
Du Pont,0.500082,0.585741,0.401645,0.333751,0.378551,1.000000,0.608584,0.496742,0.580393,0.558178
MMM,0.508054,0.561278,0.447763,0.217873,0.488355,0.608584,1.000000,0.606126,0.562761,0.602050
P&G,0.225151,0.482069,0.380622,0.216232,0.598731,0.496742,0.606126,1.000000,0.490089,0.468086
Sears,0.314567,0.648834,0.490767,0.152580,0.438786,0.580393,0.562761,0.490089,1.000000,0.602600


In [ ]:
rtn.cov()

PERMNO,Alcoa,Amex,Boeing,Chev,Coke,Du Pont,MMM,P&G,Sears,U Tech
PERMNO,,,,,,,,,,
Alcoa,0.007626,0.002753,0.002921,0.001055,0.001264,0.002976,0.002561,0.001108,0.002198,0.003578
Amex,0.002753,0.007104,0.004536,0.001575,0.002543,0.003365,0.002731,0.002290,0.004376,0.004077
Boeing,0.002921,0.004536,0.009997,0.001762,0.002642,0.002737,0.002584,0.002145,0.003926,0.005385
Chev,0.001055,0.001575,0.001762,0.007226,0.000291,0.001933,0.001069,0.001036,0.001038,0.002603
Coke,0.001264,0.002543,0.002642,0.000291,0.003576,0.001543,0.001686,0.002018,0.002100,0.002187
Du Pont,0.002976,0.003365,0.002737,0.001933,0.001543,0.004645,0.002394,0.001908,0.003165,0.003119
MMM,0.002561,0.002731,0.002584,0.001069,0.001686,0.002394,0.003332,0.001972,0.002599,0.002850
P&G,0.001108,0.002290,0.002145,0.001036,0.002018,0.001908,0.001972,0.003177,0.002210,0.002164
Sears,0.002198,0.004376,0.003926,0.001038,0.002100,0.003165,0.002599,0.002210,0.006403,0.003954


In [ ]:
#Basic Portfolio Settings
t = 50
a = 2/t  
N = len(mean)

P = matrix(a*cov)
q = matrix(-mean)
G = matrix(np.diag([-1.0 for i in range(N)]), tc='d')
h = matrix(np.array([0.0 for i in range(N)]), tc='d')
A = matrix(np.array([[1.0 for i in range(N)]]), tc='d')
b = matrix(np.array([1.0]), tc='d')

sol = solvers.qp(P,q,G,h,A,b)
solvers.options['show_progress'] = False
#CE_0 = -sol['primal objective']
w = sol['x']
CE_0 = np.dot(mean,w)[0] - 0.5*a*np.dot(np.dot(w.T, cov), w)[0][0]
print(w)

     pcost       dcost       gap    pres   dres
 0: -1.6748e-02 -1.0216e+00  1e+00  2e-16  3e+00
 1: -1.6759e-02 -3.1618e-02  1e-02  6e-17  5e-02
 2: -1.7480e-02 -2.1709e-02  4e-03  1e-16  1e-02
 3: -2.0491e-02 -2.3284e-02  3e-03  1e-16  4e-03
 4: -2.1496e-02 -2.1588e-02  9e-05  2e-16  1e-04
 5: -2.1571e-02 -2.1571e-02  9e-07  2e-16  1e-06
 6: -2.1571e-02 -2.1571e-02  9e-09  3e-16  1e-08
Optimal solution found.
[ 1.74e-07]
[ 4.85e-08]
[ 1.50e-07]
[ 1.82e-07]
[ 1.00e+00]
[ 1.86e-07]
[ 1.50e-07]
[ 1.91e-07]
[ 1.30e-07]
[ 1.41e-07]



In [ ]:
#Parameter Setting
k_list = np.arange(0.05, 0.20, 0.05)
iter_num = 100

In [ ]:
#Examine the effect of errors in means

for k in k_list:
    CEL = 0
    for n in range(iter_num):
        z = np.random.normal(size = N)*k + 1
        mean_new = np.multiply(mean, z)
    
        P = matrix(a*cov)
        q = matrix(-mean_new)
        G = matrix(np.diag([-1.0 for i in range(N)]), tc='d')
        h = matrix(np.array([0.0 for i in range(N)]), tc='d')
        A = matrix(np.array([[1.0 for i in range(N)]]), tc='d')
        b = matrix(np.array([1.0]), tc='d')

        sol = solvers.qp(P,q,G,h,A,b)
        w_new = sol['x']
        CE_x = np.dot(mean,w_new)[0] - 0.5*a*np.dot(np.dot(w_new.T, cov), w_new)[0][0]
    
        CEL += (1 - CE_x/CE_0)
    CEL /= iter_num
    print(CEL)

0.009116230461633717
0.04375105251244272
0.054268641412988795
0.08374834795362843


In [ ]:
#Examine the effect of errors in variances
for k in k_list:
    CEL = 0
    for n in range(iter_num):
        z = np.random.normal(size = N)*k + 1
        var = np.diag(cov)
        var_new = np.multiply(var, z)
        cov_new = cov + np.diag(var_new - var)
    
        P = matrix(a*cov_new)
        q = matrix(-mean)
        G = matrix(np.diag([-1.0 for i in range(N)]), tc='d')
        h = matrix(np.array([0.0 for i in range(N)]), tc='d')
        A = matrix(np.array([[1.0 for i in range(N)]]), tc='d')
        b = matrix(np.array([1.0]), tc='d')

        sol = solvers.qp(P,q,G,h,A,b)
        w_new = sol['x']
        CE_x = np.dot(mean,w_new)[0] - 0.5*a*np.dot(np.dot(w_new.T, cov), w_new)[0][0]
    
        CEL += (1 - CE_x/CE_0)
    CEL /= iter_num
    print(CEL)

3.175444573955133e-10
2.3911563840073315e-10
7.732929185877424e-10
-7.129858259347089e-10


In [ ]:
#Examine the effect of errors in covariances
for k in k_list:
    CEL = 0
    for n in range(iter_num):
        z = np.random.normal(size = N**2)*k + 1
        var = np.diag(cov)
        upper = np.triu(cov)
        cov_new = np.multiply(upper.reshape(-1), z).reshape((N,N))
        cov_new += cov_new.T
        cov_new = cov_new - np.diag(np.diag(cov_new)) + np.diag(var)
    
        P = matrix(a*cov_new)
        q = matrix(-mean)
        G = matrix(np.diag([-1.0 for i in range(N)]), tc='d')
        h = matrix(np.array([0.0 for i in range(N)]), tc='d')
        A = matrix(np.array([[1.0 for i in range(N)]]), tc='d')
        b = matrix(np.array([1.0]), tc='d')

        sol = solvers.qp(P,q,G,h,A,b)
        w_new = sol['x']
        CE_x = np.dot(mean,w_new)[0] - 0.5*a*np.dot(np.dot(w_new.T, cov), w_new)[0][0]
    
        CEL += (1 - CE_x/CE_0)
    CEL /= iter_num
    print(CEL)

6.526803097983703e-11
-2.4119522490373415e-10
-2.3440031138299844e-10
3.1026559654812045e-10
